In [ ]:
import pymysql
from pymongo import MongoClient
import pymongo

import getNYT as gn

In [ ]:
# Get connection to MySQL
try:
    conn = pymysql.connect(host='127.0.0.1',
                            user='root',
                            password='m123456',
                            db='gp3',
                            charset='utf8',
                            autocommit=True,
                            cursorclass=pymysql.cursors.DictCursor)
    c = conn.cursor()
except:
    print('Cannot connect to MySQL server!')

In [ ]:
# Get connection to MongoDB
try:
    client = MongoClient('127.0.0.1', 27017)
    db = client['MyTest']
    collection = db['news']
except:
    print('Cannot connect to Mongo server!')

In [ ]:
# Get unread url from cklist table
records = c.execute(
    """SELECT url FROM cklist WHERE source = 'NYT' LIMIT 5""")
print('Get %s records.' % (records))

urls = []
for record in c:
    urls.append(record['url'])
# print(urls)

In [ ]:
# Get article content and insert into MongoDB
okcount = 0
ngcount = 0

for url in urls:
    art_dict = gn.getArticle(url)

    if (len(art_dict['content']) > 1):
        collection.insert_one(art_dict)
        print('[%s paras inserted!] url: %s'%(len(art_dict['content']), url))
        okcount += 1
    else:
        print('\n[NG!] %s' % (art_dict['url']))
        print('The article has only %s paras.\n' % (len(art_dict['content'])))
        ngcount += 1

    c.execute("""UPDATE cklist SET tag = 'R' WHERE url = %s""", url)

print('\nCrawler finished! %s articles are inserted to MongoDB. %s are failed.\n'%(okcount, ngcount))

In [ ]:
import re
from textblob import TextBlob
from textblob import Word
from nltk.corpus import stopwords

In [ ]:
# Get connection to MongoDB@10.120.28.55
try:
    client = pymongo.MongoClient('127.0.0.1', 27017)
    db = client['MyTest']
    collection_read = db['news']
    collection_write = db['word_dict']
except:
    print('Cannot connect to MongoDB server!')

In [ ]:
# Define stopwords
with open('../TextProcessing/stop-word-list.txt', 'r', encoding='utf8') as mysw:
    swlist = stopwords.words() + mysw.read().split(',')
    mysw.close()

In [ ]:
def getWordList(art_id, contents):
    wordList = []
    for numPara in range(len(contents)):
        para_id = str(numPara + 1)
        para_wds = str(TextBlob(contents[para_id]).words.lower())

        match = re.findall('[a-z]+', para_wds)

        for word in match:
            if word not in swlist and len(word) > 2:
                finalWord = Word(word).singularize().lemmatize("v")

                word_info = {}
                word_info['_id'] = finalWord
                word_info['count'] = 1
                word_info['src'] = '%s_%s' % (art_id, para_id)
                wordList.append(word_info)
    return wordList

In [ ]:
# 傳入wordlist並更新至Mongo資料庫
def updateWord_dictCollection(wordList):
    for obj in wordList:
        if collection_write.find({'_id': obj['_id']}, {'Available': True}):
            collection_write.find_one_and_update(
                {'_id': obj['_id']},
                {'$inc': {'count': 1}, '$addToSet': {'src': obj['src']}},
                upsert=True)
        else:
            collection_write.insert_one(obj)

In [ ]:
articles = collection_read.find({'source': 'NYT', 'counted': 'false'})
for article in articles:
    try:
        contents = article['content']
        art_id = article['_id']
        wordList = getWordList(art_id, contents)
        updateWord_dictCollection(wordList)
        collection_read.update_one({'_id': art_id}, {'$set': {'counted': 'true'}})
        print('%s Update to MongoDB successfully!' % (art_id))
    except Exception as e:
        print('%s error, going to delete...' % (art_id))
        collection_read.delete_one({'_id': art_id})
print('Word count finished!')